# Analysis of NMF bicv for enterosignature regularisation choice

In [ ]:
from statistics import median, mean
import json
import matplotlib.pylab as plt
import numpy as np
from scipy.stats import sem


An important assumption for this notebook is that for each combination of 
shuffled matrix, rank, alpha value and fold, decomposition is only run once.
While currently you could set the nruns parameter to a higher value in 
`bicv_regu.py`, only the first of the runs will be taken.

## Parameters

In [ ]:
inpdir = f"./"

In [76]:
# if WRITE_OUTPUT is True, output tables will be written

WRITE_OUTPUT = True
chosen_rank = 2 # This report only carries out analysis for one rank. There
# may be more than one rank in the results though.

output = inpdir #"bicv_regu_analysis/"

Alpha values for regularisation parameters are set in the script for running bicv. 
In case they were changed, they need to be changed below too in order to prevent errors. 

In [ ]:
alphas = [0]
alphas.extend([2**x for x in range(-5,2)])
alphas.extend([5,10,50,100])
print(alphas)
#print(len(alphas))

## Read input data

In [ ]:
raw_file_evar = inpdir + "biCV_evar.json"
with open(raw_file_evar, "r") as f:
    raw_evar = json.load(f)
    
raw_file_sparsity = inpdir + "biCV_sparsity.json"
with open(raw_file_sparsity, "r") as f:
    raw_sparsity = json.load(f)
    
raw_file_l2norm = inpdir + "biCV_l2norm.json"
with open(raw_file_l2norm, "r") as f:
    raw_l2norm = json.load(f)
    
raw_file_recoerror = inpdir + "biCV_reco_error.json"
with open(raw_file_recoerror, "r") as f:
    raw_recoerror = json.load(f)

raw_file_rss = inpdir + "biCV_rss.json"
with open(raw_file_rss, "r") as f:
    raw_rss = json.load(f)
    
raw_file_cosine = inpdir + "biCV_cosine.json"
with open(raw_file_cosine, "r") as f:
    raw_cosine = json.load(f)

In [ ]:
# Determine the ranks ranks and alpha values which are in these results
ranks = sorted(int(x) for x in raw_evar[0]['0'].keys())
alphas = sorted(float(x) for x in raw_evar[0]['0'][str(ranks[0])])
(ranks, alphas)
if chosen_rank not in ranks:
    print(f"Chosen rank {chosen_rank} not in results, using {ranks[0]} instead")
    chosen_rank = ranks[0]

Each of the above objects is a list of n dictionaries corresponding to the n matrix shuffling performed in bicv.
Each dictionary is composed of several levels of dictionaries

- First level is the fold of bicv (the 9 submatrices were validation set at some point)
- Second level is the rank (number of enterosignatures)
- Third level is the regularisation ratio (alpha)
- The last level contains the keys A, B, C, D, corresponding to the parts of the matrix during bicv. A is the validation set, D is the set on which the first NMF was performed. B and C are the intermediaries. The values for the validation set A are the ones we are interested in.

In [ ]:
evars = {alpha:[] for alpha in alphas}
rss = {alpha:[] for alpha in alphas}
recoerror = {alpha:[] for alpha in alphas}
cosine = {alpha:[] for alpha in alphas}
l2norm = {alpha:[] for alpha in alphas}
sparsityH = {alpha:[] for alpha in alphas}
sparsityW = {alpha:[] for alpha in alphas}


Fill the dicts of results:

In [ ]:
#raw_evar is a list of dicts
for d in raw_evar: #run'th element of the list
    # d is a dict of dicts. Its keys are the matrices, subkeys are ranks. subsubkeys are regularisation ratios and subsubsubkeys A, B, C, D submatrices
    for mx in d:
        for regu in d[mx][str(chosen_rank)]:
            evars[float(regu)].extend(d[mx][str(chosen_rank)][regu]["A"])

#same idea for all metrics
for d in raw_cosine:
    for mx in d:
        for regu in d[mx][str(chosen_rank)]:
            cosine[float(regu)].extend(d[mx][str(chosen_rank)][regu]["A"])
            
for d in raw_rss:
    for mx in d:
        for regu in d[mx][str(chosen_rank)]:
            rss[float(regu)].extend(d[mx][str(chosen_rank)][regu]["A"])
            
for d in raw_recoerror:
    for mx in d:
        for regu in d[mx][str(chosen_rank)]:
            recoerror[float(regu)].extend(d[mx][str(chosen_rank)][regu]["A"])

for d in raw_l2norm:
    for mx in d:
        for regu in d[mx][str(chosen_rank)]:
            l2norm[float(regu)].extend(d[mx][str(chosen_rank)][regu]["A"])

for d in raw_sparsity:
    for mx in d:
        for regu in d[mx][str(chosen_rank)]:
            sparsityH[float(regu)].append(d[mx][str(chosen_rank)][regu]["A"][0][0])
            sparsityW[float(regu)].append(d[mx][str(chosen_rank)][regu]["A"][0][1])

## Write tables

In [ ]:
if WRITE_OUTPUT:
    labels, data = [*zip(*evars.items())]
    with open(f"{output}/evar.tsv", "w") as f:
        for i in range(0, len(labels)):
            for j in data[i]:
                f.write(str(labels[i]) + "\t" + str(j) + "\n")
    #
    labels, data = [*zip(*cosine.items())]
    with open(f"{output}/cosine.tsv", "w") as f:
        for i in range(0, len(labels)):
            for j in data[i]:
                f.write(str(labels[i]) + "\t" + str(j) + "\n")
    #
    labels, data = [*zip(*l2norm.items())]
    with open(f"{output}/l2norm.tsv", "w") as f:
        for i in range(0, len(labels)):
            for j in data[i]:
                f.write(str(labels[i]) + "\t" + str(j) + "\n")
    #
    labels, data = [*zip(*rss.items())]
    with open(f"{output}/rss.tsv", "w") as f:
        for i in range(0, len(labels)):
            for j in data[i]:
                f.write(str(labels[i]) + "\t" + str(j) + "\n")
    #
    labels, data = [*zip(*recoerror.items())]
    with open(f"{output}/recerror.tsv", "w") as f:
        for i in range(0, len(labels)):
            for j in data[i]:
                f.write(str(labels[i]) + "\t" + str(j) + "\n")
    #
    labels, data = [*zip(*sparsityH.items())]
    with open(f"{output}/sparsityH.tsv", "w") as f:
        for i in range(0, len(labels)):
            for j in data[i]:
                f.write(str(labels[i]) + "\t" + str(j) + "\n")
    #
    labels, data = [*zip(*sparsityW.items())]
    with open(f"{output}/sparsityW.tsv", "w") as f:
        for i in range(0, len(labels)):
            for j in data[i]:
                f.write(str(labels[i]) + "\t" + str(j) + "\n")
    

## Visualise

In [ ]:
plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
labels, data = [*zip(*evars.items())]  # 'transpose' items to parallel key, value lists

# or backwards compatible    
#labels, data = dict1.keys(), dict1.values()

plt.boxplot(data, showmeans=True)
plt.xticks(range(1, len(labels) + 1), labels)
#plt.ylim(bottom=-1, top = 1.1)
plt.show()

In [ ]:
[(alpha, mean(evars[alpha])) for alpha in evars]

In [ ]:
[(alpha, median(evars[alpha])) for alpha in evars]

In [ ]:
labels, data = [*zip(*sparsityH.items())]  # 'transpose' items to parallel key, value lists

# or backwards compatible    
#labels, data = dict1.keys(), dict1.values()

plt.boxplot(data, showmeans=True)
plt.xticks(range(1, len(labels) + 1), labels)
#plt.ylim(bottom=-1, top = 1.1)
plt.show()

In [ ]:
[(alpha, mean(sparsityH[alpha])) for alpha in sparsityH]

In [ ]:
[(alpha, median(sparsityH[alpha])) for alpha in sparsityH]

In [ ]:
labels, data = [*zip(*sparsityW.items())]  # 'transpose' items to parallel key, value lists

# or backwards compatible    
#labels, data = dict1.keys(), dict1.values()

plt.boxplot(data, showmeans=True)
plt.xticks(range(1, len(labels) + 1), labels)
#plt.ylim(bottom=-1, top = 1.1)
plt.show()

In [ ]:
[(alpha, mean(sparsityW[alpha])) for alpha in sparsityW]

In [ ]:
[(alpha, median(sparsityW[alpha])) for alpha in sparsityW]

## Get best value of regularisation coefficient

In [ ]:
evars

In [ ]:
if 0 not in evars:
    raise Exception("Regularisation of 0 must be included to set threshold")
threshold = mean(evars[0]) - np.std(evars[0], ddof=1)
print(mean(evars[0]), np.std(evars[0], ddof=1), threshold)

In [ ]:
# find the highest regularisation value greater than the threshold (obtained with regu = 0) 
regu_res = {i: None for i in range(2,11)}

means_evars = {reg: mean(evars[reg]) for reg in evars}
# values greater than threshold
v = [i for i in means_evars.values() if i > threshold]
# their associated keys
k = [list(means_evars.keys())[list(means_evars.values()).index(i)] for i in v]


print(max(k))